This is a tool to optimze day off preferences across a department. Future roadmap is to expand to integrate multiple departments.

In [35]:
import pyomo.environ as pyo
import pandas as pd
import enum as en
import math 

In [36]:
### configurable parameters ###

# valid days for days off
#vd = ['SUN', 'MON', 'TUE', 'WED', 'THU', 'FRI']
vd = [1,2,3,4,5,6]

# minimum number of people that must be at the department (does not include subs)
min_staff = 1

Sample data format:

| **columns**    | **data type** |
|----------------|---------------|
| name           | str           |
| department     | str           |
| sub_department | str           |
| first_choice   | enum          |
| second_choice  | enum          |
| third_choice   | enum          |
| other_people   | str           |
| days_over_ppl  | boolean       |



In [38]:
sample_data = pd.DataFrame(data = {'name' : ['bob', 'sally', 'charlie', 'megan', 'alice', 'zach', 'rich', 'john', 'kristen', 'mark'],
                            'department' : pd.Series(['small class']).repeat(10),
                            'sub_department' : ['atv', 'art', 'stem', 'culinary', 'art', 'bikes', 'bikes', 'culinary', 'atv', 'stem'],
                            'first_choice' : [1,3,6,5,1,3,3,4,6,3],
                            'second_choice' : [2,1,1,3,4,5,1,6,1,4],
                            'third_choice' : [4,5,2,6,3,2,4,5,5,1]}).set_index('name')


preference_data = sample_data

preference_data

,department,sub_department,first_choice,second_choice,third_choice
name,,,,,
bob,small class,atv,1,2,4
sally,small class,art,3,1,5
charlie,small class,stem,6,1,2
megan,small class,culinary,5,3,6
alice,small class,art,1,4,3
zach,small class,bikes,3,5,2
rich,small class,bikes,3,1,4
john,small class,culinary,4,6,5
kristen,small class,atv,6,1,5


In [39]:
# derivative tables
max_day_off = pd.DataFrame(preference_data.groupby('sub_department')['first_choice'].count() - min_staff).rename(columns = {'first_choice' : 'max_staff_off'})



for col in vd:
    max_day_off[col] = 0

max_day_off

,max_staff_off,SUN,MON,TUE,WED,THU,FRI
sub_department,,,,,,,
art,1,0,0,0,0,0,0
atv,1,0,0,0,0,0,0
bikes,1,0,0,0,0,0,0
culinary,1,0,0,0,0,0,0
stem,1,0,0,0,0,0,0


In [40]:
# create model

model = pyo.ConcreteModel()

# sets
model.validDays = pyo.Set(initialize = vd)
model.people = pyo.Set(initialize = sample_data.index)

# decision vars
model.D = pyo.Var(model.people, within = model.validDays)


In [49]:
# objective functions
'''
def prefer(person, assignment) :
    score = 0
    if assignment == preference_data['first_choice'][person] :
        score += 3
    elif assignment == preference_data['second_choice'][person] :
        score += 2
    elif assignment == preference_data['third_choice'][person] :
        score += 1
    else :
        score = 0
    return score
'''

def objective_function(model):
    
    return math.prod((3*int(model.D[p] == preference_data['first_choice'][p]) + 
                     2*int(model.D[p] == preference_data['second_choice'][p]) + 
                     1*int(model.D[p] == preference_data['third_choice'][p])) for p in model.people)
model.score = pyo.Objective(rule = objective_function, sense = pyo.maximize)



'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
ERROR: Rule failed when generating expression for Objective score with index
None: TypeError: int() argument must be a string, a bytes-like object or a
real number, not 'EqualityExpression'
ERROR: Constructing component 'score' from data=None failed: TypeError: int()
argument must be a string, a bytes-like object or a real number, not
'EqualityExpression'


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'EqualityExpression'

In [33]:
# subject to
def minimum_staff_rule(model, p) :
    # increase the count of the number of people in the sub-department who have a given day off
    dept = preference_data['sub_department'][p]
    day = model.D[p]
    max_day_off[day][dept] += 1

    return max_day_off[day][dept] <= max_day_off['max_staff_off'][dept]
model.minimum_staff_constraint = pyo.Constraint(model.people, rule = minimum_staff_rule)

ERROR: Rule failed when generating expression for Constraint
minimum_staff_constraint with index bob: KeyError: None
ERROR: Constructing component 'minimum_staff_constraint' from data=None
failed: KeyError: None


KeyError: None

In [23]:
opt = pyo.SolverFactory('glpk')

opt.solve(model)

RuntimeError: Attempting to solve model=unknown with unconstructed component(s)

In [48]:
def yeet(a,b):
    return int(a==b)

yeet(5,6) * 2

0